In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.xgboost import XGBoostPredictor
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timedelta
import xgboost as xgb
import matplotlib.pyplot as plt

sagemaker_role = get_execution_role()

# Specify the S3 location where the model artifacts are saved
model_artifacts_path = "s3://csv-financial-data-selazbin-crimson/modeled/model/xgboost-model"

# Create a SageMaker model from the model artifacts
xgb_predictor = XGBoostPredictor(endpoint_name="your-endpoint-name", sagemaker_session=sagemaker.Session())
xgb_predictor._model_name = "xgb_model"
xgb_predictor._model_uri = model_artifacts_path

In [ ]:
#select last data point
s3_staging_dir = "s3://appl-financial-csv/raw"
connection_string = f"awsathena+rest://:@athena.us-west-1.amazonaws.com:443/appl-db-financial?s3_staging_dir={s3_staging_dir}"
engine = create_engine(connection_string)
last_data_point = pd.read_sql('SELECT * FROM "appl-db-financial"."raw_data_output" limit 1;', engine)

# Create a new datapoint for the next day
future_date = pd.DataFrame({'Datetime': [last_data_point['Datetime'].iloc[0] + timedelta(days=1)]})

# Drop the target variable ('Close') if it's present in the features
if 'Close' in future_date.columns:
    future_date = future_date.drop('Close', axis=1)
    future_date = future_date.drop('Datetime', axis=1)

# Convert the new datapoint to XGBoost DMatrix
dnew = xgb.DMatrix(future_date, enable_categorical=True)

# Make predictions for the new datapoint
prediction_for_new_datapoint = xgb_predictor.predict(dnew)[0]

# Print the prediction for the new datapoint
print(f'Prediction for the new data point on {future_date["Datetime"].iloc[0]}: {prediction_for_new_data_point:.4f}')